In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

from art.attacks.evasion import ProjectedGradientDescent
from art.estimators.classification import TensorFlowV2Classifier
from art.utils import load_dataset

import numpy as np

from sklearn import metrics

import joblib

2024-02-26 01:13:00.419592: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-26 01:13:00.422410: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-26 01:13:00.451542: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-26 01:13:00.451602: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-26 01:13:00.452630: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
def calculate_performance_metrics(x_test, y_test, model):

    # Predictions
    y_pred = model.predict(x_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    
    report = metrics.classification_report(y_test, y_pred_classes, output_dict=True, zero_division=1)
    
    precision_macro = report['macro avg']['precision']
    precision_weighted = report['weighted avg']['precision']
    accuracy = report['accuracy']
    recall_macro = report['macro avg']['recall']
    recall_weighted = report['weighted avg']['recall']
    f1_macro = report['macro avg']['f1-score']
    f1_weighted = report['weighted avg']['f1-score']
    auc = metrics.roc_auc_score(y_test, y_pred, multi_class='ovr')

    print(f"Accuracy: {accuracy}\nAUC: {auc}")
    
    print("\nmacro")
    print(f"Precision: {precision_macro}\nRecall: {recall_macro}\nF1 Score: {f1_macro}")

    print("\nweighted")
    print(f"Precision: {precision_weighted}\nRecall: {recall_weighted}\nF1 Score: {f1_weighted}")
    print()
    
    # Confusion matrix for FNR, TNR, FPR, TPR
    cm = metrics.confusion_matrix(y_test, y_pred_classes)
    def calculate_rates(conf_matrix, class_index):
        tp = conf_matrix[class_index, class_index]
        fn = np.sum(conf_matrix[class_index, :]) - tp
        fp = np.sum(conf_matrix[:, class_index]) - tp
        tn = np.sum(conf_matrix) - (tp + fn + fp)
    
        fnr = fn / (fn + tp)
        tnr = tn / (tn + fp)
        fpr = fp / (fp + tn)
        tpr = tp / (tp + fn)
        return fnr, tnr, fpr, tpr

    # Calculate and aggregate rates
    fnrs, tnrs, fprs, tprs = [], [], [], []
    for i in range(cm.shape[0]):
        fnr, tnr, fpr, tpr = calculate_rates(cm, i)
        fnrs.append(fnr)
        tnrs.append(tnr)
        fprs.append(fpr)
        tprs.append(tpr)
    
    mean_fnr = np.mean(fnrs)
    mean_tnr = np.mean(tnrs)
    mean_fpr = np.mean(fprs)
    mean_tpr = np.mean(tprs)

    # Printing the mean metrics
    print(f"Mean FNR: {mean_fnr}\nMean TNR: {mean_tnr}\nMean FPR: {mean_fpr}\nMean TPR: {mean_tpr}")

In [3]:
x_test = np.load("/home/jovyan/X-IIoT/X-IIoT_preprocessed/x_test.npy")
x_train = np.load("/home/jovyan/X-IIoT/X-IIoT_preprocessed/x_train.npy")
x_val = np.load("/home/jovyan/X-IIoT/X-IIoT_preprocessed/x_val.npy")
y_test = np.load("/home/jovyan/X-IIoT/X-IIoT_preprocessed/y_test.npy")
y_train = np.load("/home/jovyan/X-IIoT/X-IIoT_preprocessed/y_train.npy")
y_val = np.load("/home/jovyan/X-IIoT/X-IIoT_preprocessed/y_val.npy")

In [4]:
model = joblib.load("/home/jovyan/X-IIoT/dl/dl_new/model/dnn.joblib")

2024-02-26 01:13:04.778833: W tensorflow/core/util/tensor_slice_reader.cc:98] Could not open ram://63164ffccf354a1b9dcc39f82acf9874: INVALID_ARGUMENT: ram://63164ffccf354a1b9dcc39f82acf9874 is a directory.


In [5]:
classifier = TensorFlowV2Classifier(
    model=model,
    nb_classes=19,
    input_shape=(58,),
    loss_object=tf.keras.losses.SparseCategoricalCrossentropy(),
)

In [6]:
# Initialize PGD attack
attack = ProjectedGradientDescent(estimator=classifier, eps=0.01)

# Generate adversarial test examples
adv_test_examples = attack.generate(x=x_test)

In [7]:
calculate_performance_metrics(adv_test_examples, y_test, model)

4383/4383 [==============================] - 3s 650us/step
Accuracy: 0.9202952292662056
AUC: 0.994765402709281

macro
Precision: 0.8300630131547733
Recall: 0.6906494011023545
F1 Score: 0.7151242746981334

weighted
Precision: 0.9352760550594487
Recall: 0.9202952292662056
F1 Score: 0.9209862243781489

Mean FNR: 0.3093505988976454
Mean TNR: 0.9941571854952101
Mean FPR: 0.005842814504789895
Mean TPR: 0.6906494011023545


In [8]:
# Initialize PGD attack
attack = ProjectedGradientDescent(estimator=classifier, eps=0.05)

# Generate adversarial test examples
adv_test_examples = attack.generate(x=x_test)

In [9]:
calculate_performance_metrics(adv_test_examples, y_test, model)

4383/4383 [==============================] - 3s 677us/step
Accuracy: 0.45160807245239964
AUC: 0.884119340969312

macro
Precision: 0.2558579609065153
Recall: 0.20851031821783167
F1 Score: 0.19057933865542012

weighted
Precision: 0.6020934861904934
Recall: 0.45160807245239964
F1 Score: 0.4979770074820043

Mean FNR: 0.7914896817821684
Mean TNR: 0.961032665857742
Mean FPR: 0.03896733414225808
Mean TPR: 0.20851031821783167


In [7]:
# Initialize PGD attack
attack = ProjectedGradientDescent(estimator=classifier, eps=0.1)

# Generate adversarial test examples
adv_test_examples = attack.generate(x=x_test)

In [8]:
calculate_performance_metrics(adv_test_examples, y_test, model)

4383/4383 [==============================] - 3s 581us/step
Accuracy: 0.21610211794908366
AUC: 0.6726825105569056

macro
Precision: 0.08565149032997321
Recall: 0.07488933151289774
F1 Score: 0.06351891583536287

weighted
Precision: 0.3037621358648797
Recall: 0.21610211794908366
F1 Score: 0.24016758660320067

Mean FNR: 0.9251106684871021
Mean TNR: 0.9393236460917895
Mean FPR: 0.0606763539082103
Mean TPR: 0.07488933151289774


In [9]:
# Initialize PGD attack
attack = ProjectedGradientDescent(estimator=classifier, eps=0.15)

# Generate adversarial test examples
adv_test_examples = attack.generate(x=x_test)

In [10]:
calculate_performance_metrics(adv_test_examples, y_test, model)

4383/4383 [==============================] - 3s 604us/step
Accuracy: 0.11247949796762462
AUC: 0.5220936845746863

macro
Precision: 0.032656792057214855
Recall: 0.03935302257986238
F1 Score: 0.025618923546649745

weighted
Precision: 0.1618216647330128
Recall: 0.11247949796762462
F1 Score: 0.12529910491418056

Mean FNR: 0.9606469774201376
Mean TNR: 0.9315907065343848
Mean FPR: 0.06840929346561513
Mean TPR: 0.03935302257986238


In [11]:
# Initialize PGD attack
attack = ProjectedGradientDescent(estimator=classifier, eps=0.2)

# Generate adversarial test examples
adv_test_examples = attack.generate(x=x_test)

In [12]:
calculate_performance_metrics(adv_test_examples, y_test, model)

4383/4383 [==============================] - 3s 594us/step
Accuracy: 0.043842259145689226
AUC: 0.41649558907344547

macro
Precision: 0.012036888601757967
Recall: 0.026452517375230477
F1 Score: 0.012052257260721358

weighted
Precision: 0.05758320547694477
Recall: 0.043842259145689226
F1 Score: 0.043400977891719046

Mean FNR: 0.9735474826247695
Mean TNR: 0.926666054995035
Mean FPR: 0.07333394500496505
Mean TPR: 0.026452517375230477


In [13]:
# Initialize PGD attack
attack = ProjectedGradientDescent(estimator=classifier, eps=0.25)

# Generate adversarial test examples
adv_test_examples = attack.generate(x=x_test)

In [14]:
calculate_performance_metrics(adv_test_examples, y_test, model)

4383/4383 [==============================] - 3s 595us/step
Accuracy: 0.022634243742423163
AUC: 0.39687406274656045

macro
Precision: 0.058910933454235564
Recall: 0.02226986900665553
F1 Score: 0.008034653324362958

weighted
Precision: 0.023501849731730154
Recall: 0.022634243742423163
F1 Score: 0.01717537224531967

Mean FNR: 0.9777301309933446
Mean TNR: 0.9251932094883142
Mean FPR: 0.07480679051168586
Mean TPR: 0.02226986900665553


In [15]:
# Initialize PGD attack
attack = ProjectedGradientDescent(estimator=classifier, eps=0.3)

# Generate adversarial test examples
adv_test_examples = attack.generate(x=x_test)

In [16]:
calculate_performance_metrics(adv_test_examples, y_test, model)

4383/4383 [==============================] - 3s 592us/step
Accuracy: 0.016651215859659133
AUC: 0.3462443276953082

macro
Precision: 0.057684522512523866
Recall: 0.019883735906578694
F1 Score: 0.006975360772884151

weighted
Precision: 0.015167758877872695
Recall: 0.016651215859659133
F1 Score: 0.010977232711232987

Mean FNR: 0.9801162640934213
Mean TNR: 0.9256961292610913
Mean FPR: 0.07430387073890858
Mean TPR: 0.019883735906578694
